In [1]:
import numpy as np
import open3d as o3d

# Intrinsic parameters (from PrimeSense Carmine)
fx = fy = 525.0
cx = 319.5
cy = 239.5

def depth_to_point_cloud(depth_img):
    h, w = depth_img.shape
    i, j = np.meshgrid(np.arange(w), np.arange(h))

    # Remove invalid depth values
    mask = depth_img > 0
    z = depth_img[mask].astype(np.float32) / 1000.0  # Convert mm to meters
    x = (i[mask] - cx) * z / fx
    y = (j[mask] - cy) * z / fy

    points = np.stack((x, y, z), axis=-1)
    return points, mask

def point_cloud_to_mesh(points, mask):
    h, w = mask.shape
    indices = -np.ones((h, w), dtype=int)
    indices[mask] = np.arange(points.shape[0])

    triangles = []
    for y in range(h - 1):
        for x in range(w - 1):
            ids = [indices[y, x], indices[y, x+1], indices[y+1, x], indices[y+1, x+1]]
            if all(i >= 0 for i in [ids[0], ids[1], ids[2]]):
                triangles.append([ids[0], ids[1], ids[2]])
            if all(i >= 0 for i in [ids[1], ids[3], ids[2]]):
                triangles.append([ids[1], ids[3], ids[2]])

    return np.array(triangles)

def main():
    # Load sample depth image using Open3D
    redwood_data = o3d.data.SampleRedwoodRGBDImages()
    depth_o3d = o3d.io.read_image(redwood_data.depth_paths[1])

    # Convert Open3D depth image to numpy array
    depth_img = np.asarray(depth_o3d)

    # Proceed with your existing pipeline
    print("Depth image shape:", depth_img.shape)

    points, mask = depth_to_point_cloud(depth_img)
    triangles = point_cloud_to_mesh(points, mask)

    # Create Open3D mesh
    mesh = o3d.geometry.TriangleMesh()
    mesh.vertices = o3d.utility.Vector3dVector(points)
    mesh.triangles = o3d.utility.Vector3iVector(triangles)
    mesh.compute_vertex_normals()

    # Visualize the mesh
    o3d.visualization.draw_geometries([mesh])

    # Save the mesh to a file
    o3d.io.write_triangle_mesh("output_mesh.ply", mesh)

if __name__ == "__main__":
    main()


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] Downloading https://github.com/isl-org/open3d_downloads/releases/download/20220301-data/SampleRedwoodRGBDImages.zip
[Open3D INFO] Downloaded to C:\Users\Utente/open3d_data/download/SampleRedwoodRGBDImages/SampleRedwoodRGBDImages.zip
[Open3D INFO] Created directory C:\Users\Utente/open3d_data/extract/SampleRedwoodRGBDImages.
[Open3D INFO] Extracting C:\Users\Utente/open3d_data/download/SampleRedwoodRGBDImages/SampleRedwoodRGBDImages.zip.
[Open3D INFO] Extracted to C:\Users\Utente/open3d_data/extract/SampleRedwoodRGBDImages.
Depth image shape: (480, 640)


In [2]:
import open3d as o3d

# Load the saved mesh
mesh = o3d.io.read_triangle_mesh("output_mesh.ply")

# Visualize using WebVisualizer
o3d.visualization.draw_geometries([mesh], window_name="Mesh")

### If like my laptop your system is not supporting showing ply because of prerequirments, so we can check the ply results by this site:

https://imagetostl.com/view-ply-online